In [14]:
import os
import sys
import json
import pandas as pd
from pandas import json_normalize

In [8]:
!pip install -r requirements.txt

In [11]:
#Prepare the labels in coco format
paths=['downloads/SoccerPass/SoccerPass-COCO/train.json']

dfs= []
for path in paths:
    # Load the COCO data from the JSON file
    with open(path) as f:
        coco_data = json.load(f)

    # Normalize the data into a DataFrame
    images_df = json_normalize(coco_data['images'])
    annotations_df = json_normalize(coco_data['annotations'])

    images_df=images_df.drop(columns=['license', 'flickr_url','coco_url', 'date_captured'])
    annotations_df=annotations_df.drop(columns=['segmentation', 'area','attributes.occluded','attributes.rotation','iscrowd'])
    annotations_df[['x', 'y', 'width', 'height']]=pd.DataFrame(annotations_df['bbox'].to_list())

    merged_df = pd.merge(annotations_df, images_df, right_on='id',left_on="image_id", how='left')

    merged_df['x'] = merged_df['x'] /merged_df['width_y']
    merged_df['width_x'] = merged_df['width_x'] /merged_df['width_y']
    merged_df['y'] = merged_df['y'] /merged_df['height_y']
    merged_df['height_x'] = merged_df['height_x'] /merged_df['height_y']


    merged_df['new_id'] = merged_df.groupby('image_id').cumcount() + 1
    merged_df=merged_df.drop(columns=['id_x', 'image_id', 'bbox', 'id_y', 'width_y', 'height_y'])

    new_columns = {'category_id': 'class',
                'attributes.Team': 'team',
                'attributes.Ball_Possesion': 'Ball_Possesion',
                'attributes.Ball_Reciever': 'Ball_Reciever',
                'width_x': 'w',
                'height_x': 'h',
                'file_name': 'image_name',
                'new_id': 'OID'}

    merged_df = merged_df.rename(columns=new_columns)
    merged_df.loc[merged_df['class']==1,'class']=0
    merged_df.loc[merged_df['class']==2,'class']=1
    # merged_df.to_csv("mer.csv",index=False)
    dfs.append(merged_df)

label_df=pd.concat(dfs)
display(label_df)


,class,Ball_Possesion,Ball_Reciever,team,x,y,w,h,image_name,OID
0,0,0,0,1,0.967187,0.258333,0.030469,0.123611,image_vics_1342.png,1
1,1,NaN,NaN,NaN,0.332031,0.586111,0.011719,0.020833,image_vics_1342.png,2
2,0,0,0,0,0.725000,0.054167,0.020313,0.062500,image_vics_1342.png,3
3,0,0,0,0,0.191406,0.098611,0.018750,0.065278,image_vics_1342.png,4
4,0,0,0,1,0.571875,0.331944,0.030469,0.119444,image_vics_1342.png,5
...,...,...,...,...,...,...,...,...,...,...
16306,0,0,0,0,0.315625,0.425000,0.022656,0.094444,image_vimp_689.png,14
16307,0,0,0,2,0.412734,0.311111,0.017734,0.094444,image_vimp_689.png,15
16308,0,0,1,0,0.225000,0.194444,0.025000,0.059722,image_vimp_689.png,16
16309,0,0,0,0,0.317969,0.362500,0.018750,0.070833,image_vimp_689.png,17


In [18]:
sys.path.append('/home/vb2573/FootballPassPrediction/PerspectiveTransform')

In [23]:
from elements.perspective_transform import Perspective_Transform
from elements.assets import transform_matrix, detect_color
from elements.args import Args

import torch
import os
import cv2
import numpy as np
import sys

#Load Arguments for perspective transform model
opt = Args()

# Load model
perspective_transform = Perspective_Transform(opt)

paths = ['downloads/SoccerPass/SoccerPass-COCO/images/train']

OSError: /opt/conda/lib/python3.7/site-packages/torch/lib/../../nvidia/cublas/lib/libcublas.so.11: symbol cublasLtGetStatusString version libcublasLt.so.11 not defined in file libcublasLt.so.11 with link time reference

In [24]:
def generate_perspective_transform(model, image_path->List, yolo_df):

    groups = yolo_df.groupby("image_name")

    # loop through each group and apply the code snippet to each image
    for image_name, group in groups:


        image_path = None
        for path in paths:
            temp_path = os.path.join(path, image_name)
            if os.path.exists(temp_path):
                image_path = temp_path
                break
        
        if image_path is None:
            raise ValueError(f"Image file not found: {image_name}")
    

        cap = cv2.imread(image_path)

        w = cap.shape[1] #1280
        h = cap.shape[0] #720

        main_frame = cap.copy()

        M, warped_image = perspective_transform.homography_matrix(main_frame)


        for index, row in group.iterrows():
            x_center = row['x'] #if x is center
            y_center = row['y'] + (row['h']/2)

            coords = transform_matrix(M, (x_center*w, y_center*h), (h, w), (68, 105))

            yolo_df.loc[index, 'gnd_x'] = coords[0]
            yolo_df.loc[index, 'gnd_y'] = coords[1]
        
        return yolo_df


SyntaxError: invalid syntax (3474061640.py, line 1)

In [25]:
with torch.no_grad():
    transformed_df = generate_perspective_transform(perspective_transform, paths, label_df)

NameError: name 'torch' is not defined